In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
x = np.linspace(-2,2, 100)
y = np.linspace(-2,2, 100)
X, Y = np.meshgrid(x, y)

def FLU(x, y, c):
    return np.clip(1 - x**2 - (-y-c*x**2+c/4)**2 , 0, None)
def flu(x, y):
    return FLU(x, y, c=0.5)

Z = flu(X, Y)

LEVELS = [-0.2, 0.1, 0.45, 0.7, 0.9, 1.1]

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(Z[::-1], cmap='hot_r', interpolation='hermite', extent=(-2,2,-2,2))
plt.xticks([])
plt.yticks([])
plt.title('Optimal fluence map')

plt.subplot(1, 2, 2)
plt.contour(X, Y, Z, levels=LEVELS, cmap='hot_r', linewidths=5)
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.gca().set_aspect('equal')
plt.xticks([])
plt.yticks([])
plt.title('Level sets of fluence map')

# plt.savefig('_fluence_map_discretization.pdf', bbox_inches='tight', pad_inches=0.1)
# plt.savefig('_fluence_map_discretization.svg', bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
LEAF_WIDTH = 0.25
LEAF_LENGTH = 1.25

In [ ]:
def get_leaf(direction, leaf_number, level, func, reverse=False):
    global LEAF_WIDTH, LEAF_LENGTH
    direction_length = np.sqrt(direction[0]**2 + direction[1]**2)
    direction = (direction[0]/direction_length, direction[1]/direction_length)
    x = direction[1] * LEAF_WIDTH * leaf_number
    x0 = x
    y = -direction[0] * LEAF_WIDTH * leaf_number
    y0 = y
    while x*x + y*y < 10:
        x -= direction[0]*0.01
        y -= direction[1]*0.01
    dx, dy = direction
    dx *= 0.01
    dy *= 0.01
    while func(x,y) < level and x*x + y*y < 100:
        x += dx
        y += dy
    x -= dx
    y -= dy
    if x*x + y*y >= 90:
        x = x0
        y = y0
    rx = x
    ry = y
    y = y - LEAF_WIDTH/2
    if not reverse:
        sx = -LEAF_LENGTH
    else:
        sx = LEAF_LENGTH
    sy = LEAF_WIDTH
    a = np.arctan2(direction[1], direction[0]) * (180/np.pi)
    a %= 180
    return plt.Rectangle((x, y), sx, sy, \
                         edgecolor='black', facecolor='grey', linewidth=2, \
                         rotation_point=(rx,ry), angle=a)


In [ ]:
def get_leaf_pair(direction, leaf_number, level, func):
    leaf1 = get_leaf(direction, leaf_number, level, func)
    reverse_direction = (-direction[0], -direction[1])
    leaf2 = get_leaf(reverse_direction, leaf_number, level, func, reverse=True)
    return leaf1, leaf2

In [ ]:
def plot_leaves_level(fluence_function, level, levels=None, direction=(1,0), number_leaves=5):
    Z = fluence_function(X, Y)
    plt.contour(X, Y, Z, levels, cmap='hot_r', linewidths=3, alpha=0.8)
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.gca().set_aspect('equal')
    plt.xticks([])
    plt.yticks([])
    for leaf_number in range(-number_leaves, number_leaves+1):
        r1, r2 = get_leaf_pair(direction, leaf_number, level, fluence_function)
        plt.gca().add_patch(r1)
        plt.gca().add_patch(r2)

In [ ]:
def plot_leaves_levels(fluence_function, direction=(1,0), number_leaves=5, levels=None, filename=None):
    global LEVELS
    if levels is None:
        levels = LEVELS
    N = len(levels)-2
    plt.figure(figsize=(4*N, 4))
    for i,level in enumerate(levels[1:-1]):
        plt.subplot(1, N, i+1)
        plot_leaves_level(fluence_function, level, levels, direction, number_leaves)
        plt.title(f'Level set {i+1}')
    plt.suptitle('Leaves positions')
    if filename is not None:
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.1)
    plt.show()

plot_leaves_levels(flu, number_leaves=6)
# plot_leaves_levels(flu, number_leaves=6, filename='_level_set_matching_with_leaves.pdf')
# plot_leaves_levels(flu, number_leaves=6, filename='_level_set_matching_with_leaves.svg')

In [ ]:
# just to show off
plot_leaves_levels(flu, (1,2), number_leaves=6)

In [ ]:
# slightly non-convex fluence map
def flu0(x, y):
    return FLU(x, y, c=1.6)


In [ ]:
def plot_leaves_directions(fluence_function, directions=[(1,0), (0,1)], number_leaves=5, levels=[-2, 0.3, 2], filename=None):
    N = len(directions)
    plt.figure(figsize=(4*(N+1), 4))
    plt.subplot(1, N+1, 1)
    Z = fluence_function(X, Y)
    plt.imshow(Z[::-1], cmap='hot_r', interpolation='hermite', extent=(-2,2,-2,2))
    plt.xticks([])
    plt.yticks([])
    plt.title('Optimal fluence map')
    level = levels[1]
    for i,direction in enumerate(directions):
        plt.subplot(1, N+1, i+2)
        plot_leaves_level(fluence_function, level, levels, direction, number_leaves)
        angle = round(np.arctan2(direction[1], direction[0]) * (180/np.pi))
        plt.title(f'Colimator angle: {angle}°')
    if filename is not None:
        plt.savefig(filename, bbox_inches='tight', pad_inches=0.1)
    plt.show()

plot_leaves_directions(flu0, number_leaves=6)
# plot_leaves_directions(flu0, number_leaves=6, filename='_leaves_angle.pdf')
# plot_leaves_directions(flu0, number_leaves=6, filename='_leaves_angle.svg')

In [ ]:
# slightly non-convex fluence map
def flu1(x, y):
    return flu0(x, y) + flu0(y, x)

plot_leaves_directions(flu1, directions=[(1,0), (0,1), (1,1)], number_leaves=6)
# plot_leaves_directions(flu1, directions=[(1,0), (0,1), (1,1)], number_leaves=6, filename='_leaves_angle_bis.pdf')
# plot_leaves_directions(flu1, directions=[(1,0), (0,1), (1,1)], number_leaves=6, filename='_leaves_angle_bis.svg')

In [ ]:
def flu2(x, y):
    def dist(x1, y1, x2, y2):
        return np.sqrt((x1-x2)**2 + (y1-y2)**2)
    def bump(x):
        return np.exp(-x**2)
    c = 1.25
    d1 = dist(x, y, -1, -np.sqrt(3)/3 -0.2)
    d2 = dist(x, y, 1, -np.sqrt(3)/3 -0.2)
    d3 = dist(x, y, 0, np.sqrt(3)*2/3 -0.2)
    d4 = dist(x, y, 0, -0.2)
    z = bump(c*d1) + bump(c*d2) + bump(c*d3) + 0.25 * bump(2*d4)
    return z

plt.figure(figsize=(15, 5) )
Z = flu2(X, Y)

plt.subplot(1, 3, 1)
plt.imshow(Z[::-1], cmap='hot_r', interpolation='hermite', extent=(-2,2,-2,2))
plt.xticks([])
plt.yticks([])
plt.title('Optimal fluence map')

plt.subplot(1, 3, 2)
plt.contour(X, Y, Z, levels=np.linspace(0, 1.25, 20), cmap='hot_r', linewidths=2)
plt.xticks([])
plt.yticks([])
plt.gca().set_aspect('equal')
plt.title('Level sets of the fluence map')

plt.subplot(1, 3, 3)
lvl = flu2(0,0)-0.01
plt.contour(X, Y, Z, levels=[0, 0.5*lvl, lvl, 1.5*lvl, 1.25], cmap='hot_r', linewidths=5)
plt.xticks([])
plt.yticks([])
plt.gca().set_aspect('equal')
plt.title('Problematic level set of the fluence map')

# plt.savefig('_fluence_map_discretization_non_convex.pdf', bbox_inches='tight', pad_inches=0.1)
# plt.savefig('_fluence_map_discretization_non_convex.svg', bbox_inches='tight', pad_inches=0.1)
plt.show()


In [ ]:
LEAF_LENGTH = 2
directions = [(1,0), (3**0.5, 1), (1,1), (1,3**0.5), (0,1)]
plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, 0.5*lvl, 1.2])
plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, lvl, 1.2])
plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, 1.5*lvl, 1.2])

# plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, 0.5*lvl, 1.2], filename='_leaves_angle_ter0.pdf')
# plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, lvl, 1.2], filename='_leaves_angle_ter.pdf')
# plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, 1.5*lvl, 1.2], filename='_leaves_angle_ter2.pdf')

# plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, 0.5*lvl, 1.2], filename='_leaves_angle_ter0.svg')
# plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, lvl, 1.2], filename='_leaves_angle_ter.svg')
# plot_leaves_directions(flu2, directions, number_leaves=9, levels=[0, 1.5*lvl, 1.2], filename='_leaves_angle_ter2.svg')